# 1. Water Consumer Data test models with KFold

## 1.1. Initialization

In [17]:
from IPython.display import display
import pandas as pd

df_train = pd.read_csv('src/train.csv')
df_train

,Year,Month,Consumer_type,Consumption,Consumer_number,Installation_zone
0,2013,1,domestic,0,MOGV36480546611521,Installation_zone 1
1,2013,1,industrial,5,BECS02817768252637,Installation_zone 2
2,2013,1,domestic,6,VRFW65577141436242,Installation_zone 2
3,2013,1,domestic,1,QLLI18662653137621,Installation_zone 2
4,2013,1,domestic,13,HYUO61823402850645,Installation_zone 2
...,...,...,...,...,...,...
329970,2020,12,domestic,0,ZSOE03393580274296,Installation_zone 2
329971,2020,12,rural domestic,0,RVZG94535208799772,Installation_zone 47
329972,2020,12,rural domestic,0,DWGA69725679861554,Installation_zone 7
329973,2020,12,rural domestic,0,VJDC45865594518045,Installation_zone 32


## 2. Training Tests

### 2.0. Pré processing

#### 2.0.1 Random Seed

In [18]:
from numpy import random

SEED = 42
random.seed(SEED)

#### 2.0.2. Open file

In [19]:
from datetime import datetime

df = df_train.copy()
map_names = {
    'Year': 'year',
    'Month': 'month',
    'Consumer_type': 'consumer_type',
    'Consumption': 'consumption',
    'Consumer_number': 'consumer_number',
    'Installation_zone': 'installation_zone',
}
df = df.rename(columns=map_names)
df = df.drop(columns=['consumer_number'])
df.installation_zone = df.installation_zone.str.replace('Installation_zone ', '').astype(int)
# df['age'] = datetime.now().year - df.year
# df = df.drop(columns=['year'])

Y = df.consumer_type
X = df.drop(columns=['consumer_type'])

X

,year,month,consumption,installation_zone
0,2013,1,0,1
1,2013,1,5,2
2,2013,1,6,2
3,2013,1,1,2
4,2013,1,13,2
...,...,...,...,...
329970,2020,12,0,2
329971,2020,12,0,47
329972,2020,12,0,7
329973,2020,12,0,32


#### 2.0.3. Split test and train

In [20]:

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, stratify=Y)

print('Train size: ', len(x_train))
print('Test size: ', len(x_test))


Train size:  247481
Test size:  82494


#### 2.0.4. Scaler

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


#### 2.0.5. KFold cross validation

In [22]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=10, shuffle=True)

### 2.1 Dummy Classifier

In [23]:
# dummy classifier
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='most_frequent')

dummy.fit(x_train, y_train)
y_pred = dummy.predict(x_test)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

dummy_accuracy = accuracy_score(y_test, y_pred)*100
dummy_precision = precision_score(y_test, y_pred, average='macro')*100
dummy_recall = recall_score(y_test, y_pred, average='macro')*100
dummy_f1 = f1_score(y_test, y_pred, average='macro')*100

print(f'Accuracy: {dummy_accuracy:.2f}%')
print(f'Precision: {dummy_precision:.2f}%')
print(f'Recall: {dummy_recall:.2f}%')
print(f'F1: {dummy_f1:.2f}%')


/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 71.57%
Precision: 10.22%
Recall: 14.29%
F1: 11.92%


### 2.2. Logistic Regression

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

# Create and train the logistic regression model
model = LogisticRegression(solver='liblinear')

# Create the pipeline
pipeline = Pipeline([
    ('scaler', scaler),
    ('model', model)
])

logic_regression_results = cross_validate(pipeline, X, Y, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])

logic_regression_accuracy = logic_regression_results['test_accuracy'].mean()*100
logic_regression_precision = logic_regression_results['test_precision_macro'].mean()*100
logic_regression_recall = logic_regression_results['test_recall_macro'].mean()*100
logic_regression_f1 = logic_regression_results['test_f1_macro'].mean()*100

print(f'Accuracy: {logic_regression_accuracy:.2f}%')
print(f'Precision: {logic_regression_precision:.2f}%')
print(f'Recall: {logic_regression_recall:.2f}%')
print(f'F1: {logic_regression_f1:.2f}%')


/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Accuracy: 88.66%
Precision: 37.49%
Recall: 27.11%
F1: 26.60%


### 2.3. SVM

In [25]:
# from sklearn.svm import SVC

# model = SVC(kernel='linear')
# model.fit(x_train, y_train)

# y_pred = model.predict(x_test)

# svm_accuracy = accuracy_score(y_test, y_pred)*100
# svm_precision = precision_score(y_test, y_pred, average='macro')*100
# svm_recall = recall_score(y_test, y_pred, average='macro')*100
# svm_f1 = f1_score(y_test, y_pred, average='macro')*100

# print(f'Accuracy: {svm_accuracy:.2f}%')
# print(f'Precision: {svm_precision:.2f}%')
# print(f'Recall: {svm_recall:.2f}%')
# print(f'F1: {svm_f1:.2f}%')


### 2.4. Decision Tree

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

model = DecisionTreeClassifier(max_depth=3)

pipeline = Pipeline([
    ('scaler', scaler),
    ('model', model)
])
decision_tree_results = cross_validate(pipeline, X, Y, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])

decision_tree_accuracy = decision_tree_results['test_accuracy'].mean()*100  
decision_tree_precision = decision_tree_results['test_precision_macro'].mean()*100
decision_tree_recall = decision_tree_results['test_recall_macro'].mean()*100
decision_tree_f1 = decision_tree_results['test_f1_macro'].mean()*100

print(f'Accuracy: {decision_tree_accuracy:.2f}%')
print(f'Precision: {decision_tree_precision:.2f}%')
print(f'Recall: {decision_tree_recall:.2f}%')
print(f'F1: {decision_tree_f1:.2f}%')


/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Accuracy: 90.93%
Precision: 39.59%
Recall: 30.88%
F1: 31.46%


### 2.5. Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

model = RandomForestClassifier(max_depth=3)
pipeline = Pipeline([
    ('scaler', scaler),
    ('model', model)
])
random_forest_results = cross_validate(pipeline, X, Y, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])

random_forest_accuracy = random_forest_results['test_accuracy'].mean()*100
random_forest_precision = random_forest_results['test_precision_macro'].mean()*100
random_forest_recall = random_forest_results['test_recall_macro'].mean()*100
random_forest_f1 = random_forest_results['test_f1_macro'].mean()*100

print(f'Accuracy: {random_forest_accuracy:.2f}%')
print(f'Precision: {random_forest_precision:.2f}%')
print(f'Recall: {random_forest_recall:.2f}%')
print(f'F1: {random_forest_f1:.2f}%')


/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Accuracy: 90.67%
Precision: 26.36%
Recall: 28.56%
F1: 27.41%


### 2.6. KNeighbors Classifier

In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

model = KNeighborsClassifier(n_neighbors=5)
pipeline = Pipeline([
    ('scaler', scaler),
    ('model', model)
])
knn_results = cross_validate(pipeline, X, Y, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])

knn_accuracy = knn_results['test_accuracy'].mean()*100
knn_precision = knn_results['test_precision_macro'].mean()*100
knn_recall = knn_results['test_recall_macro'].mean()*100
knn_f1 = knn_results['test_f1_macro'].mean()*100

print(f'Accuracy: {knn_accuracy:.2f}%')
print(f'Precision: {knn_precision:.2f}%')
print(f'Recall: {knn_recall:.2f}%')
print(f'F1: {knn_f1:.2f}%')


/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Accuracy: 89.88%
Precision: 41.05%
Recall: 32.32%
F1: 33.82%


### 2.7. Naive Bayes

In [29]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

model = GaussianNB()
pipeline = Pipeline([
    ('scaler', scaler),
    ('model', model)
])
naive_bayes_results = cross_validate(pipeline, X, Y, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])

naive_bayes_accuracy = naive_bayes_results['test_accuracy'].mean()*100
naive_bayes_precision = naive_bayes_results['test_precision_macro'].mean()*100
naive_bayes_recall = naive_bayes_results['test_recall_macro'].mean()*100
naive_bayes_f1 = naive_bayes_results['test_f1_macro'].mean()*100

print(f'Accuracy: {naive_bayes_accuracy:.2f}%')
print(f'Precision: {naive_bayes_precision:.2f}%')
print(f'Recall: {naive_bayes_recall:.2f}%')
print(f'F1: {naive_bayes_f1:.2f}%')

/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/run/media/guedes/HD/ipb/ml4ds/water-consumption/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Accuracy: 89.39%
Precision: 32.79%
Recall: 28.37%
F1: 28.28%


## 3. Comparing

In [30]:
data = {
    'method': ['dummy', 'logistic regression', 'decision tree', 'random forest', 'knn', 'naive bayes'],
    'accuracy': [dummy_accuracy, logic_regression_accuracy, decision_tree_accuracy, random_forest_accuracy, knn_accuracy, naive_bayes_accuracy],
    'precision': [dummy_precision, logic_regression_precision, decision_tree_precision, random_forest_precision, knn_precision, naive_bayes_precision],
    'recall': [dummy_recall, logic_regression_recall, decision_tree_recall, random_forest_recall, knn_recall, naive_bayes_recall],
    'f1': [dummy_f1, logic_regression_f1, decision_tree_f1, random_forest_f1, knn_f1, naive_bayes_f1]
}

df_results = pd.DataFrame(data)
df_results

,method,accuracy,precision,recall,f1
0,dummy,71.571266,10.224467,14.285714,11.918623
1,logistic regression,88.658837,37.486322,27.109282,26.599060
2,decision tree,90.934768,39.589516,30.880811,31.456964
3,random forest,90.668081,26.363131,28.560329,27.411885
4,knn,89.884688,41.053924,32.316185,33.823336
5,naive bayes,89.388893,32.792084,28.369028,28.276973
